# Machine Learning - Master IEF Parcours Quantitatif
## Prediction of Sharpe ratio for blends of quantitative strategies par Napoleon X (2019)
### Giovanni MANCHE et Antonin DEVALLAND

Lien vers le GitHub du projet : https://github.com/GiovanniManche/MachineLearning272

## Table des matières
J'arrive pas à la mettre   + faudra suivre l'outline donnée dans Moodle

## 1. Sujet

### 1.1. Contexte

Le sujet que nous avons choisi de traiter concerne la prédiction du ratio de Sharpe d'un mix de stratégies quantitatives, *Challenge Data du Collège de France* proposé par Napoleon X.
Lien : https://challengedata.ens.fr/participants/challenges/13/

Napoleon Crypto (NC), vranche de Napoleon X, est une entreprise spécialisée dans la conception de solutions d'investissement quantitatives, basées sur des algorithmes. Ayant développé plusieurs stratégies performantes, NC cherche à optimiser l'allocation entre ces stratégies pour maximiser les performances. L'objectif de ce problème est donc d'aider à la construction d'un **mélange optimal de stratégies quantitatives d'investissement** à partir de l'ensemble des stratégies soumises. Trouver la meilleure allocation parmi les stratégies quantitatives chaque semaine revient à déterminer la combinaison maximisant le ratio de Sharpe (sur les 5 prochains jours de trading). Avec $Lr_{i,s}$ les rendements logarithmiques d'une stratégie $i$ à un temps $s$, le ratio de Sharpe annualisé objectif est calculé selon la formule suivante (il est légèrement modifié pour éviter une volatilité trop proche de 0) : 
$$
S_t^*(w_1, \ldots, w_7) = \frac{\frac{252}{5} \sum_{i=1}^{7} w_i \times \left( \sum_{s=t+1}^{t+5} Lr_{i,s} \right)}{Max \left( \sqrt{252 \times \sum_{i=1}^{7} \sum_{j=1}^{7} w_i w_j \sum_{s=t-20}^{t+5} (Lr_{i,s} - \bar{Lr_i})(Lr_{j,s} - \bar{Lr_j})} ; 0.005 \right)}
$$

L'enjeu est donc, finalement, de prédire le ratio de Sharpe $S^*$ d'une combinaison donnée de poids et d'optimiser en tenant compte du compromis rendement / volatilité. 

### 1.2. Description des données
Pour répondre à ce sujet, Napoleon X nous fournit des données concernant 7 stratégies quantitatives qu'ils ont mis en place. Pour chaque stratégie et chaque observation ("sample") nous sont fournit les poids alloués à chaque stratégie, la valeur de chacune des stratégies sur 21 jours de trading, et les valeurs de 3 instruments financiers (inconnus) sur 21 jours de trading. Dans un même sample, les jours de trading sont identiques, mais entre les samples, ces jours sont, logiquement, différents. Nous avons à notre disposition trois fichiers CSV contenant : 
- les données d'entraînement (10 000 observations par variable). Les poids alloués aux stratégies, les valeurs des stratégies et les valeurs des instruments financiers représentent nos *features*.
- les données de test (4 450 observations par variable)
- les valeurs du label (ratio de Sharpe, 10 000 valeurs cibles)

Les données d'entraînement et de test sont organisées de la façon suivante : 
- la première colonne correspond à l'ID de l'observation,
- les 7 colonnes suivantes correspondent aux poids alloués aux 7 stratégies qui nous sont proposées,
- les colonnes suivantes correspondent aux valeurs des autres *features* sur 21 jours de trading (séries temporelles). On a donc 21 colonnes par stratégie, et par produit financiers, soit un total de : 
$$
\underbrace{1}_{ID} + \underbrace{7}_{\text{Poids}} + \underbrace{7 \times 21}_{\text{Valeurs des stratégies sur 21 jours}} + \underbrace{3 \times 21}_{\text{Valeurs des instruments sur 21 jours}} = 218 \text{ colonnes}  
$$
Les valeurs des stratégies et des instruments financiers sont normalisées à 100 au départ ($lag_{21}$). Les poids sont strictement positifs et somment à 1. 

### 1.3. Description du benchmark

Chaque modèle proposé produira *in fine* des valeurs de ratio de Sharpe selon la formule présentée plus haut, et sera comparée aux vraies valeurs du ratio de Sharpe. Avant la comparaison, une fonction de lissage des extrêmes est appliquée aux résultats : 
$$
f(x) = sign(x) \times e^{-\frac{1}{abs(x)}}
$$
Une fois les résultats lissés, la métrique / fonction de perte retenue est la moyenne des erreurs absolues : 
$$
d(y, \hat{y}) = \frac{1}{N}\sum_{i=1}^N |f(y_i) - f(\hat{y_i})|
$$

## 2. Problématique

Ce sujet permet ainsi de faire appel au machine learning pour proposer une réponse à une problématique absolument cruciale pour tout gestionnaire : comment optimiser l'allocation des différentes stratégies d'investissement quantitatives pour maximiser le ratio de Sharpe, ou dit plus simplement, comment ajuster le niveau d'*asset under management* au sein d'une même société de gestion entre les différentes stratégies pour maximiser la performance.

Le passage du problème financier au problème de machine learning se pose formellement. Avec $X \in mathcal{X}$ les *features* (poids, valeurs des stratégies, valeurs des instruments) et $Y \in mathcal{Y}$ les labels (ici, uniquement le ratio de Sharpe), on cherche $h_{\theta}^*$ la meilleure approximation de la fonction de mapping $\phi : \mathcal{X} \rightarrow \mathcal{Y}$ définie telle que 
$$ y_i = \phi(x_i) + \varepsilon_i \ \forall i = {1,...,n}$$

Ici, le label (ratio de Sharpe) peut *théoriquement* prendre ses valeurs dans $\mathbb{R}$, nous sommes donc face à un **problème de régression**.

Il faudra poser l'espace des hypothèses + la fonction hypothèse. La fonction de coût est déjà donnée.

## Import des bibliothèques nécessaires 

In [ ]:
import pandas as pd

## Partie I : Récupération et préparation des données

Le fichier CSV fournit pour l'entraînement pesait trop lourd (plus de 35 000 Ko, alors que la limite pour déposer les données sur GitHub est de 25 000 Ko). Nous l'avons donc converti en un fichier xlsx., moins lourd, au prix d'un temps d'importation via Python allongé.

In [ ]:
train_set: pd.DataFrame = pd.read_excel("Data/Train set.xlsx")
test_set: pd.DataFrame = pd.read_csv("Data/Test set.csv")
target_values: pd.DataFrame = pd.read_csv("Data/Target values.csv")

Nous effectuons quelques vérifications pour s'assurer que le jeu de données est utilisable : 
- gestion des données manquantes
- gestion des données dupliquées
- uniformisation des formats de données
- gestion des outliers

Tout d'abord, on s'assure que les fichiers ne présentent pas de lignes dupliquées.

In [ ]:
print(f"Les dimensions du train set initial sont {train_set.shape} et celles du nouveau train set sont de {train_set.drop_duplicates().shape}.")
print(f"Les dimensions du train set initial sont {test_set.shape} et celles du nouveau train set sont de {test_set.drop_duplicates().shape}.")
print(f"Les dimensions du train set initial sont {target_values.shape} et celles du nouveau train set sont de {target_values.drop_duplicates().shape}.")

Les dimensions du train set initial sont (10000, 218) et celles du nouveau train set sont de (10000, 218)
Les dimensions du train set initial sont (4450, 218) et celles du nouveau train set sont de (4450, 218)
Les dimensions du train set initial sont (10000, 2) et celles du nouveau train set sont de (10000, 2)


Ensuite, on vérifie l'absence de données manquantes.

In [10]:
print(f"Le fichier contenant les données d'entraînement contient {train_set.isnull().sum().sum()} données manquantes.")
print(f"Le fichier contenant les données de test contient {test_set.isnull().sum().sum()} données manquantes.")
print(f"Le fichier contenant les valeurs cibles contient {target_values.isnull().sum().sum()} données manquantes.")

Le fichier contenant les données d'entraînement contient 0 données manquantes.
Le fichier contenant les données de test contient 0 données manquantes.
Le fichier contenant les valeurs cibles contient 0 données manquantes.


À toutes fins utiles, on s'assure que nos données sont toutes de type float. En effet, le train set et test set sont composés de valeurs décimales (parts, valeurs des stratégies, valeurs des instruments financiers), il n'y a donc pas de raison qu'elles soient autre chose que de type float.

In [15]:
train_set = train_set.astype(float)
target_values = target_values.astype(float)
test_set = test_set.astype(float)

In [65]:
target_values

,ID,Target
0,0.0,-12.007941
1,1.0,2.294867
2,2.0,0.652308
3,3.0,2.412364
4,4.0,8.517471
...,...,...
9995,9995.0,1.455430
9996,9996.0,-1.532292
9997,9997.0,3.496214
9998,9998.0,2.764467


On peut désormais faire quelques vérifications sur les valeurs : 
- les valeurs des parts doivent être comprises entre 0 et 1 (répartition de l'AUM entre différentes stratégies)
- la somme des parts doit être égale à 1 (tolérance de $10^{-4}$)
- les valeurs initiales des stratégies et des instruments financiers doivent être égales à 100 (initialisation)


In [ ]:
# Tests sur les poids
weights_train = train_set.iloc[:,1:8]
weights_test = test_set.iloc[:, 1:8]
# Ensemble d'entraînement
print(f"Dans le set d'entraînement, {((weights_train < 0) | (weights_train > 1)).sum().sum()} samples présentent de poids supérieurs à 1 ou inférieurs à 0")
print(f"Dans le set d'entraînement, {abs((weights_train.sum(axis=1) - 1)> 0.0001).sum()} samples ont une somme totale de poids différents de 1")
# Ensemble de test
print(f"Dans le set de test, {((weights_test < 0) | (weights_test > 1)).sum().sum()} samples présentent de poids supérieurs à 1 ou inférieurs à 0")
print(f"Dans le set de test, {abs((weights_test.sum(axis=1) - 1)> 0.0001).sum()} samples ont une somme totale de poids différents de 1")


Dans le set d'entraînement, 0 samples présentent de poids supérieurs à 1 ou inférieurs à 0
Dans le set d'entraînement, 0 samples ont une somme totale de poids différents de 1
Dans le set de test, 0 samples présentent de poids supérieurs à 1 ou inférieurs à 0
Dans le set de test, 0 samples ont une somme totale de poids différents de 1


In [64]:
# Tests sur les valeurs initiales
init_train = train_set.filter(like = 'lag_20')
init_test = test_set.filter(like = 'lag_20')
print(f"Dans le set d'entraînement, {((init_train != 100)).sum().sum()} samples présentent des valeurs initiales de portefeuilles ou d'instruments différentes de 100")
print(f"Dans le set de test, {((init_test != 100)).sum().sum()} samples présentent des valeurs initiales de portefeuilles ou d'instruments différentes de 100")

Dans le set d'entraînement, 0 samples présentent des valeurs initiales de portefeuilles ou d'instruments différentes de 100
Dans le set de test, 0 samples présentent des valeurs initiales de portefeuilles ou d'instruments différentes de 100


## Analyse exploratoire